# Tensorflow/Keras notes
This notebook is a collection of my notes and code on how to run tensorflow

In [1]:
%matplotlib inline
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

In [31]:
print(tf.VERSION)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


In [2]:
print(tf.VERSION)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


This website (https://medium.com/tensorflow/what-are-symbolic-and-imperative-apis-in-tensorflow-2-0-dfccecb01021) has a great overview on the different ways of interacting with TensorFlow. The main ways are as follows:

1. Symbolic or declarative style
    - Sequential
        - This is the way that might fit our intuition better or how we visually think of neural networks as stacking different layers. The highest level Keras API takes this approach.
    - Functional
        - This allows you to create your network in DAG format. This provides more flexibility
2. Imperative model
    - This allows you to subclass the models. Provides even more flexility.
3. Low-level tensor flow or theano
    - This is much lower level and probably not recommended for ease of use.


### Sequential model
This is where we stack layers together. Use the xor data

In [112]:
# Create data
X = np.array([[0.0,0.0],[0.0,1.0],[1.0,1.0],[1.0,0.0]])
y = np.array([0.0,1.0,0.0,1.0])

X = np.array([[0.0,0.0]])
y = np.array([0.0])

#### Initialize model

In [33]:
model = tf.keras.Sequential()

#### Add layers in
This adds a regular neural network layer with 2 nodes, activation function "relu" and initializes all the weights and bias to 1.

In [34]:
model.add(layers.Dense(2,activation="relu", kernel_initializer=tf.keras.initializers.constant(1.0),
                      bias_initializer=tf.keras.initializers.constant(1.0)))
model.add(layers.Dense(1, kernel_initializer=tf.keras.initializers.constant(1.0),
                      bias_initializer=tf.keras.initializers.constant(1.0)))

#### Compile model

In [35]:
model.compile(optimizer=tf.train.GradientDescentOptimizer(0.01), loss='mse', metrics=['mse'])


#### Define callbacks (or ways of tracking your model)

In [72]:
class Histories(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.weights = [self.model.get_weights()]
 
    def on_train_end(self, logs={}):
        return
 
    def on_epoch_begin(self, epoch, logs={}):
        return
 
    def on_epoch_end(self, epoch, logs={}):
        return
 
    def on_batch_begin(self, batch, logs={}):
        return
 
    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.weights.append(self.model.get_weights())
        return

In [73]:
history = Histories()

#### Train model

In [38]:
from tensorflow.keras.callbacks import TensorBoard
from time import time

In [39]:
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

In [40]:
model.fit(X, y, batch_size=1, epochs=1, callbacks=[history, tensorboard])

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
1/1 [==============================] - 0s 75ms/sample - loss: 9.0000 - mean_squared_error: 9.0000


In [41]:
tensorboard --logdir=test/ --host localhost --port 8088

SyntaxError: invalid syntax (<ipython-input-41-1504d88355fc>, line 1)

In [42]:
len(history.weights)

2

In [43]:
history.losses

[9.0]

In [44]:
history.weights

[[array([[1., 1.],
         [1., 1.]], dtype=float32),
  array([1., 1.], dtype=float32),
  array([[1.],
         [1.]], dtype=float32),
  array([1.], dtype=float32)],
 [array([[1., 1.],
         [1., 1.]], dtype=float32),
  array([0.94, 0.94], dtype=float32),
  array([[0.94],
         [0.94]], dtype=float32),
  array([0.94], dtype=float32)]]

In [85]:
model.fit?

## Understanding the Tensor Flow session paradigm

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
with tf.Session() as sess:
    a = tf.constant(15, name="a")
    b = tf.constant(5, name="b")
    prod = tf.multiply(a,b,name="multiply")
    add = tf.add(a,b, name="add")
    res = tf.divide(prod, add, name="divide")
    
    out = sess.run(res)
 
    print(out)
tf.summary.FileWriter('test',sess.graph)

#### Type this in command line to see graph in tensor board

In [ ]:
tensorboard --logdir=test/ --host localhost --port 8088

In [ ]:


input_x = tf.placeholder(tf.float32, [None, 2], name='input_x')    
dense1 = layers.Dense(2, activation='relu', kernel_initializer=tf.keras.initializers.constant(2.0),bias_initializer=tf.keras.initializers.constant(1.0))
y = dense1(input_x)

weights = dense1.get_weights()

In [ ]:
dense1.get_output_at?

In [ ]:
dense1

In [ ]:
input_x = tf.placeholder(tf.float32, [None, 2], name='input_x') 

In [ ]:
input_x

In [ ]:
dense1 = layers.Dense(2, activation='relu')

In [ ]:
weights = dense1.get_weights()

In [ ]:
weights

In [ ]:
y = dense1(input_x)

In [ ]:
inputs = layers.Input(shape=(2,))

In [ ]:
inputs

## Basic linear regression using Keras

In [71]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import datasets
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer

from matplotlib import pyplot as plt

%matplotlib inline

#### Import data

In [72]:
diabetes = datasets.load_diabetes()

In [84]:
X_train = diabetes.data[:,2]

In [89]:
y_train = diabetes.target

In [93]:
data = pd.DataFrame(X_train)
data.columns = ['X']
data['y'] = y_train

In [5]:
dat = sm.datasets.get_rdataset("Guerry", "HistData").data

In [48]:
X_train = dat.Literacy.values.reshape(-1,1)
y_train = dat.Lottery.values.reshape(-1,1)

In [50]:
sc = StandardScaler()
x_ = sc.fit_transform(X_train)
y_ = sc.fit_transform(y_train)


#### Build neural network

In [97]:
lm_model = tf.keras.Sequential()
lm_model.add(layers.Dense(1,  activation='linear'))

In [98]:
lm_model.compile(optimizer=tf.keras.optimizers.SGD(), loss='mse', metrics=['mse'])

In [99]:
class Histories(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.weights = [self.model.get_weights()]
 
    def on_train_end(self, logs={}):
        return
 
    def on_epoch_begin(self, epoch, logs={}):
        return
 
    def on_epoch_end(self, epoch, logs={}):
        return
 
    def on_batch_begin(self, batch, logs={}):
        return
 
    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.weights.append(self.model.get_weights())
        return

In [100]:
history = Histories()

In [103]:
lm_model.fit(X_train,y_train, batch_size=1, epochs=200, callbacks=[history])

Epoch 1/200
442/442 [==============================] - 0s 951us/sample - loss: 7252.8267 - mean_squared_error: 7252.8247
Epoch 2/200
442/442 [==============================] - 0s 959us/sample - loss: 5887.1713 - mean_squared_error: 5887.1729
Epoch 3/200
442/442 [==============================] - 0s 992us/sample - loss: 5761.2633 - mean_squared_error: 5761.2632
Epoch 4/200
442/442 [==============================] - 0s 958us/sample - loss: 5763.6651 - mean_squared_error: 5763.6670
Epoch 5/200
442/442 [==============================] - 0s 951us/sample - loss: 5677.2324 - mean_squared_error: 5677.2300
Epoch 6/200
442/442 [==============================] - 0s 951us/sample - loss: 5624.1870 - mean_squared_error: 5624.1846
Epoch 7/200
442/442 [==============================] - 0s 960us/sample - loss: 5527.3964 - mean_squared_error: 5527.3955
Epoch 8/200
442/442 [==============================] - 0s 959us/sample - loss: 5465.8121 - mean_squared_error: 5465.8101
Epoch 9/200
442/442 [===========

442/442 [==============================] - 0s 982us/sample - loss: 3957.1238 - mean_squared_error: 3957.1255
Epoch 136/200
442/442 [==============================] - 0s 906us/sample - loss: 3928.3680 - mean_squared_error: 3928.3667
Epoch 137/200
442/442 [==============================] - 0s 953us/sample - loss: 3915.3114 - mean_squared_error: 3915.3125
Epoch 138/200
442/442 [==============================] - 0s 963us/sample - loss: 3945.8115 - mean_squared_error: 3945.8113
Epoch 139/200
442/442 [==============================] - 0s 956us/sample - loss: 3957.7852 - mean_squared_error: 3957.7861
Epoch 140/200
442/442 [==============================] - 0s 987us/sample - loss: 3960.7040 - mean_squared_error: 3960.7051
Epoch 141/200
442/442 [==============================] - 1s 2ms/sample - loss: 3946.8061 - mean_squared_error: 3946.8074
Epoch 142/200
442/442 [==============================] - 0s 978us/sample - loss: 3961.6257 - mean_squared_error: 3961.6257
Epoch 143/200
442/442 [=========

In [104]:
history.weights[-1]

[array([[932.3852]], dtype=float32), array([154.4339], dtype=float32)]

#### Statsmodels for comparison

In [63]:
dat['Lottery_transformed'] = y_
dat['Literacy_transformed'] = x_

In [95]:
results = smf.ols('y ~ X', data=data).fit()

In [96]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.344
Model:                            OLS   Adj. R-squared:                  0.342
Method:                 Least Squares   F-statistic:                     230.7
Date:                Wed, 12 Jun 2019   Prob (F-statistic):           3.47e-42
Time:                        10:33:33   Log-Likelihood:                -2454.0
No. Observations:                 442   AIC:                             4912.
Df Residuals:                     440   BIC:                             4920.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    152.1335      2.974     51.162      0.0

In [1]:
import tensorflow_visualize


In [2]:
import pandas as pd

In [3]:
test = pd.DataFrame({'test':[1,3], 'test2':[3,4]})

In [4]:
meta_file = "test_metadata.tsv"
output_path = "test_viz"
vec_file = 'model.vec'

In [5]:
tensorflow_visualize.tf_visualize?

In [5]:
tensorflow_visualize.tf_visualize(test, meta_file, output_path, labels=None)

Instructions for updating:
Colocations handled automatically by placer.
Num nodes: 2
Run `tensorboard --logdir=test_viz --port xxxx --host 0.0.0.0` to run visualize result on tensorboard
